In [51]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

In [58]:
df = pd.read_csv('./../Naive_Bayes/tweets/allLabeledTweets.csv')

In [65]:
df['label'].value_counts()

0    253
2    136
1     87
Name: label, dtype: int64

In [66]:
df_0 = df[df['label']==0]
df_1 = df[df['label']==1]
df_2 = df[df['label']==2]

In [67]:
df_0_downsampled = df_0.sample(df_1.shape[0])
df_2_downsampled = df_2.sample(df_1.shape[0])

df_balanced = pd.concat([df_0_downsampled, df_2_downsampled, df_1])
df_balanced['label'].value_counts()

0    87
2    87
1    87
Name: label, dtype: int64

In [68]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balanced['message_lowercase'],df_balanced['label'], stratify=df_balanced['label'])

In [69]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4")

In [70]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [71]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [72]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [73]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_1 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [74]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [75]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
7/7 [==============================] - 70s 7s/step - loss: 0.5807 - accuracy: 0.3590 - precision: 0.6986 - recall: 0.7846
Epoch 2/10
7/7 [==============================] - 51s 7s/step - loss: 0.3733 - accuracy: 0.3385 - precision: 0.6701 - recall: 1.0000
Epoch 3/10
7/7 [==============================] - 49s 7s/step - loss: 0.2275 - accuracy: 0.3385 - precision: 0.6701 - recall: 1.0000
Epoch 4/10
7/7 [==============================] - 49s 7s/step - loss: 0.1480 - accuracy: 0.3385 - precision: 0.6701 - recall: 1.0000
Epoch 5/10
7/7 [==============================] - 49s 7s/step - loss: 0.1175 - accuracy: 0.3385 - precision: 0.6701 - recall: 1.0000
Epoch 6/10
7/7 [==============================] - 49s 7s/step - loss: 0.0994 - accuracy: 0.3385 - precision: 0.6701 - recall: 1.0000
Epoch 7/10
7/7 [==============================] - 50s 7s/step - loss: 0.0640 - accuracy: 0.3333 - precision: 0.6667 - recall: 1.0000
Epoch 8/10
7/7 [==============================] - 49s 7s/step - loss:

In [82]:
X_train, X_test, y_train, y_test = train_test_split(df_balanced['clean_message_no_freq_stopwords'],df_balanced['label'], stratify=df_balanced['label'])

In [83]:
model2 = tf.keras.Model(inputs=[text_input], outputs = [l])
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_1 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [84]:
model2.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [85]:
model2.fit(X_train, y_train, epochs=10)

Epoch 1/10
7/7 [==============================] - 55s 7s/step - loss: 0.4170 - accuracy: 0.3333 - precision: 0.6667 - recall: 1.0000
Epoch 2/10
7/7 [==============================] - 49s 7s/step - loss: 0.3361 - accuracy: 0.3333 - precision: 0.6667 - recall: 1.0000
Epoch 3/10
7/7 [==============================] - 49s 7s/step - loss: 0.2611 - accuracy: 0.3333 - precision: 0.6667 - recall: 1.0000
Epoch 4/10
7/7 [==============================] - 49s 7s/step - loss: 0.2101 - accuracy: 0.3333 - precision: 0.6667 - recall: 1.0000
Epoch 5/10
7/7 [==============================] - 49s 7s/step - loss: 0.1580 - accuracy: 0.3333 - precision: 0.6667 - recall: 1.0000
Epoch 6/10
7/7 [==============================] - 53s 7s/step - loss: 0.1226 - accuracy: 0.3333 - precision: 0.6667 - recall: 1.0000
Epoch 7/10
7/7 [==============================] - 57s 8s/step - loss: 0.0844 - accuracy: 0.3333 - precision: 0.6667 - recall: 1.0000
Epoch 8/10
7/7 [==============================] - 56s 8s/step - loss:

In [88]:
predict = model2.predict(X_test)

In [93]:
from sklearn.metrics import accuracy_score, classification_report

predict = predict.flatten()

print(classification_report(y_test, predict))

print(accuracy_score(y_test, predict))

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets